# **Notebook de transfert des donnees du site nba.com depuis le site ou un fichier intermediaire**

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys, os, re, time #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct

from datetime import datetime
from TelechargementDonnees import JoueursSiteNba, DriverFirefox, gererCookie,simplifierNomJoueur, JourneeSiteNba, Blessures
from TeleversementJourneeBdd import JourneeBdd, insererBlessesInconnusMatchBefore, miseAJourBlessesBdd, insererBlessesInconnusPasMatchBefore
from sqlalchemy import MetaData

In [35]:
pd.set_option('display.max_rows', 500)

# 1. Données de joueurs 
Il s'agit de transfere dans le cas des donnees de joueuer telechargees en amont, ou d'ajouter des joueurs non presents au moment d'insertion de match

>## 1.1 Enumerations des types de position
A creer a la main

In [27]:
with ct.ConnexionBdd('basket','maison') as c : 
    pd.DataFrame({'id_position_terrain':['F', 'G', 'C', 'C-F', 'F-G', 'G-F', 'F-C','NC'], 
              'nom_position_terrain':['Forward ; 3 ; ailier', 'Guard ; 1 ou 2  ; meneur ou arriere', 
                                      'Center ; 5 ; Pivot' ,'Power Forward ; 4 ; ailier Fort', 
                                      'Shooting Guard ; 2 ; arriere', 'Shooting Guard ; 2 ; arriere',
                                      'Power Forward ; 4 ; ailier Fort', 'Non connue']}).to_sql(
        'enum_position_terrain', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

> ## 1.2 Donnees chargees depuis la page internet
Là ce n'est que le premier import, on ne gère pas la mise à jour

In [79]:
#creation de la classe si import depuis le net
pagejoueuer=JoueursSiteNba(urlPageJoueurs='https://www.nba.com/player/1630173/precious-achiuwa',typeExport='One' ) #joueur unique
pagejoueuer=JoueursSiteNba #joueurs multiples

In [80]:
pagejoueuer.dfJoueurs

In [39]:
#si import depuis json
Joueur=pd.read_json(r'C:\Users\martin.schoreisz\git\Basket\Basket\data\Joueurs_saison_2020-2021\joueurs.json')
#correction format date
Joueur.date_entree_nba=Joueur.date_entree_nba.apply(lambda x : pd.to_datetime(x,unit='ms'))
Joueur.date_naissance=Joueur.date_naissance.apply(lambda x : pd.to_datetime(x,unit='ms'))
#si besoin correction position bizrre car inconnues sur le net (normalenet pris en cmpte en amont dans nouvelle versions)
Joueur.loc[~Joueur.id_position_terrain.isin(['F', 'G', 'C', 'C-F', 'F-G', 'G-F', 'F-C','NC']),'id_position_terrain']='NC'

In [40]:
with ct.ConnexionBdd('basket','maison') as c : 
    #remmplacer pagejoueuer.dfJoueurs par Joueur si import Json
    Joueur[['nom','id_position_terrain','taille', 'poids','date_entree_nba','nom_simple']].to_sql(
        'joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 2. Données d'équipe
On utilise les données de joueurs pour faire la liste des equipes, et on complete avec les divisions et conférence, puis on transfere

In [41]:
dicoNom={'MIA':'Miami Heat','MIL':'Milwaukee Bucks','NOP':'New-Orleans Pelicans','SAS':'San-Antonio Spurs',
'PHX':'Phoenix','MEM':'Memphis Grizzlies',    'BKN':'Brooklyn Nets','ORL':'Orlando Magic','LAL':'LosAngeles Lakers','POR':'Portland TrailBlazzers','TOR':'Toronto Raptors','CHI':'Chicago Bulls',
'OKC':'Oklahoma City Thunder','WAS':'Washington Wizards','UTA':'Utah Jazz','SAC':'Sacramento Kings','CHA':'Charlotte Hornets','NYK':'New-York Knicks','DEN':'Denver Nuggets',
'LAC':'Los Angeles Clippers','GSW':'Golden State Warriors','MIN':'Minessota TimberWolves','DET':'Detroit Pistons','DAL':'Dallas Mavericks',
'IND':'Indiana Pacers','ATL':'Atlanta Hawks','CLE':'Cleveland Cavaliers','PHI':'Philadelphia Sixers','BOS':'Boston Celtics','HOU':'Houston Rockets'}
list_equipe=dicoNom.keys()

In [42]:
dicoDivision={'Central Division': ['MIL', 'CHI', 'IND', 'CLE', 'DET'],'Atlantic Division': ['BOS', 'BKN', 'NYK', 'TOR', 
'PHI'],'Southeast Division' : ['MIA', 'ORL', 'WAS', 'CHA', 'ATL'],'Southwest Division': ['SAS', 'HOU', 'DAL', 'NOP', 
'MEM'],'Northwest Division' : ['DEN', 'MIN', 'OKC', 'POR', 'UTA'],'Pacific Division' : ['LAC', 'LAL', 'PHX', 'GSW', 'SAC']}
listDivision=[k for e in list_equipe for k,v in dicoDivision.items()  if e in v] 

In [43]:
dicoConference={'Ouest' : ['SAS', 'HOU', 'DAL', 'NOP', 'MEM','DEN', 'MIN', 'OKC', 'POR', 'UTA','LAC', 'LAL', 'PHX', 
    'GSW', 'SAC'],'Est':['MIL', 'CHI', 'IND', 'CLE', 'DET','BOS', 'BKN', 'NYK', 'TOR', 'PHI','MIA', 
                         'ORL', 'WAS', 'CHA', 'ATL']}

listConference=[k for e in list_equipe for k,v in dicoConference.items()  if e in v] 
listNom=[v for k,v in dicoNom.items() for e in list_equipe if k==e] 

In [44]:
with ct.ConnexionBdd('basket','boulot') as c : 
    pd.DataFrame({'id_equipe':list_equipe,'nom_equipe': listNom,
              'conference':listConference, 'division':listDivision}).to_sql(
        'equipe', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 3. Données de match
Lecture et transfert des données du site vers la bdd

In [ ]:
for j in [d.strftime('%Y-%m-%d') for d in pd.date_range(start='02/08/2021',periods=13)] : 
    print(j)
    journee=JourneeBdd(j)
    journee.creerAttributsGlobaux()
    journee.exporterVersBdd()

2021-02-08
0 https://www.nba.com/game/hou-vs-cha-0022000367/box-score#box-score
1 https://www.nba.com/game/was-vs-chi-0022000368/box-score#box-score
2 https://www.nba.com/game/tor-vs-mem-0022000369/box-score#box-score
3 https://www.nba.com/game/min-vs-dal-0022000370/box-score#box-score
4 https://www.nba.com/game/gsw-vs-sas-0022000371/box-score#box-score
5 https://www.nba.com/game/cle-vs-phx-0022000372/box-score#box-score
6 https://www.nba.com/game/mil-vs-den-0022000373/box-score#box-score
7 https://www.nba.com/game/okc-vs-lal-0022000374/box-score#box-score
2021-02-09
0 https://www.nba.com/game/bkn-vs-det-0022000375/box-score#box-score
1 https://www.nba.com/game/nyk-vs-mia-0022000376/box-score#box-score
2 https://www.nba.com/game/hou-vs-nop-0022000377/box-score#box-score
3 https://www.nba.com/game/gsw-vs-sas-0022000378/box-score#box-score
4 https://www.nba.com/game/bos-vs-uta-0022000379/box-score#box-score
5 https://www.nba.com/game/orl-vs-por-0022000380/box-score#box-score
6 https://ww

# 4. exemple de blessure
le joueur Markelle Fultz s'est fait une rupture des ligaments croisé, exemple d'insertion dans la table blessure

In [ ]:
nomJoueurBlesse='Markelle Fultz'
idTypeBlessure=1
dateBlessure='2021-06-01'
requeteBlessure = f"""insert into donnees_source.blessure(id_joueur,id_type_blessure,date_blessure) 
             select id_joueur,{idTypeBlessure},{dateBlessure} from donnees_source.joueur where lower(nom)=lower{nomJoueurBlesse}"""
with ct.ConnexionBdd('basket','maison') as c : 
    c.execute(c.sqlAlchemyConn,requeteBlessure)

In [46]:
#dfsInjuries=Blessures().dfInjuries
with ct.ConnexionBdd('basket','maison') as c : 
    #miseAJourBlessesBdd(dfsInjuries,c.sqlAlchemyConn)
    #insererBlessesInconnusMatchBefore(dfsInjuries,c.sqlAlchemyConn)
    insererBlessesInconnusPasMatchBefore(dfsInjuries,c.sqlAlchemyConn)